# Serial Connection

In [2]:
:dep serialport

In [3]:
use serialport::{available_ports, SerialPortType};
use std::io::{self, Write};
use serialport::SerialPort;
use std::time::Duration;

# Open Serial Port

In [4]:
let ports = serialport::available_ports().expect("No ports found!");
for p in ports {
    println!("{}", p.port_name);
}


COM16


()

In [60]:
let mut port = serialport::new("COM16", 115_200).timeout(Duration::from_millis(10))    
    .open().expect("Failed to open port");

# Cmd

In [6]:
:dep corncobs

In [7]:
fn set_period_duty_cycle(pcnt:u16, dcnt:u16) -> [u8;6]{      
    let k:[u8;4] = [pcnt.to_be_bytes(), dcnt.to_be_bytes()].concat().try_into().unwrap();
    let mut encoded = [0u8;6];
    corncobs::encode_buf(&k, &mut encoded[..]);
    encoded
}

## read firmware version

In [8]:
port.write(&[3,1,2,0])

Ok(4)

In [9]:
let rx_bytes = port.bytes_to_read()?;
rx_bytes

5

In [10]:
let mut buf: [u8; 200] = [0; 200];
if rx_bytes > 0 {
    port.read(&mut buf[0..rx_bytes as usize])?;
    let mut output  = [0u8;400];
    let k = corncobs::decode_buf(&buf, &mut output).unwrap();
    println!("{:?}",output[0..k].to_vec());
}

[2, 0, 0]


()

## 8MHz classE PWM

each pcnt and dcnt = 8ns; 8MHz --> 125ns --> 15.6 cnt

In [67]:
port.write(&set_period_duty_cycle(15,8))

Ok(6)

# classE circuit design

Cshunt = open;
Cseries = 68pF;
L = 7.4uH